<a href="https://colab.research.google.com/github/ratsgo/nlpbook/blob/master/examples/document_classification/deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ratsnlp

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 778kB 6.2MB/s 
     |████████████████████████████████| 317kB 8.4MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 12.4MB/s 
     |████████████████████████████████| 890kB 18.3MB/s 
     |████████████████████████████████| 3.0MB 23.4MB/s 
     |████████████████████████████████| 276kB 47.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b5f8d3cc5ac8b653f24001df729a657385a03602a095d8891741d2009252b7e2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=3cfb5c146a35359bc57cbab6fe578530bfcb19892bbc983e57f07e588d38161e
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built sac

In [ ]:
import torch
from ratsnlp import nlpbook
from google.colab import drive
from ratsnlp.nlpbook.classification import get_web_service_app
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

In [ ]:
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
args = nlpbook.DeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_checkpoint_path="/gdrive/My Drive/nlpbook/checkpoint-cls/epoch=1.ckpt",
    downstream_task_name="document-classification",
    max_seq_length=128,
)

In [ ]:
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_path,
    map_location=torch.device("cpu")
)

In [ ]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [ ]:
model = BertForSequenceClassification(pretrained_model_config)

In [ ]:
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        logits, = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

In [ ]:
app = get_web_service_app(inference_fn)

In [ ]:
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cabae0d38ae2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Sep/2020 09:28:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 09:28:07] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Sep/2020 09:28:09] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 09:29:20] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 09:29:22] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 09:29:25] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2020 09:29:28] "POST /api HTTP/1.1" 200 -
